# Running MD simulations

Running vanilla MD simulations on protein-ligand complexes can provide information about the dynamics of a given ligand inside the binding site, and can be used to assess the quality and stability of a proposed binding mode.

Below we will demonstrate an example of running MD simulations on a protein ligand complex, following docking. We will be running this example on an ASAP target, the SARS-CoV-2 nsp3 Mac1 macrodomain that  removes ADP ribose from viral and host cell proteins. The removal of this post-translational modification reduces the inflammatory and antiviral responses to infection — facilitating replication.

See [here](https://asapdiscovery.notion.site/Targeting-Opportunity-SARS-CoV-2-nsp3-Mac1-macrodomain-47af24638b994e8ba786303ec743926e) for more information on Mac1. 

In [5]:
# import some dependencies
from asapdiscovery.data.testing.test_resources import fetch_test_file
from asapdiscovery.docking.docking import DockingInputPair
from asapdiscovery.docking.openeye import POSITDocker
from asapdiscovery.data.schema.complex import Complex, PreppedComplex
from asapdiscovery.data.schema.ligand import Ligand
from asapdiscovery.simulation.simulate import VanillaMDSimulator

# Docking an arbitrary ligand to Mac1
Lets fetch an example structure of Mac1, hosted in the ASAP testing repository.

In [6]:
protein = fetch_test_file("SARS2_Mac1A-A1013.pdb") # fetch a PDB file from the test suite, in this case a PDB from the COVID MOONSHOT.

Now lets turn it into a complex object (see the tutorial on base level ASAP abstractions for more info)

In [24]:
# make a complex 
mac1_complex = Complex.from_pdb(protein, ligand_kwargs={"compound_name": "A1013"}, target_kwargs={"target_name": "SARS2_Mac1A"})

Now we will create a new ligand to dock into the structure

In [8]:
# make the ligand we want to dock, a simple alkane
ligand = Ligand.from_smiles("CCCCCCC", compound_name="alkane")


DPI: 0.12, RFree: 0.28, Resolution: 1.48
Processing BU # 1 with title: ---_LIG, chains AB


We will run protein prep for this structure

In [ ]:
# prepare our structure
prepped_mac1_complex_complex = PreppedComplex.from_complex(mac1_complex)
# pair it up with the ligand we want to dock.
docking_input_pair = DockingInputPair(complex=prepped_mac1_complex_complex, ligand=ligand)


Now we dock it to our protein

In [17]:
# run OpenEye POSIT docking,
docker = POSITDocker(use_omega=False)
results = docker.dock([docking_input_pair], use_dask=False)



{'docking-confidence-POSIT': 0.019999999552965164, '_POSIT_method': 'FRED'}


In [18]:
print(results[0].posed_ligand.tags)

{'docking-confidence-POSIT': 0.019999999552965164, '_POSIT_method': 'FRED'}


## Vizualise the docked pose
Lets vizualise our results! For more information on vizualisations, see the vizualisation tutorial

In [14]:
# create a visualization factory. 
from asapdiscovery.dataviz.html_viz import HTMLVisualizer

html_vizualizer = HTMLVisualizer(
        target="SARS-CoV-2-Mac1",
        color_method="subpockets",
        align=True,
        output_dir="html",
        write_to_disk=True,
    )
vizs_from_docked =  html_vizualizer.visualize(inputs=results, outpaths=["from_docked.html"], use_dask=False)

2024-05-03 17:46:36,574 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-05-03 17:46:36,574 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-05-03 17:46:36,574 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-05-03 17:46:36,574 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of tmp_complex.pdb
2024-05-03 17:46:36,676 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in /var/folders/f5/0zcc5b7570jc40ws28tqdp740000gn/T/tmpr1owqkki/


In [15]:
from IPython.display import IFrame
IFrame(vizs_from_docked["html_path_pose"][0], 1000, 1000)

## Running an MD simulation

Great, our pose looks good! Lets use the VanillaMDSimulator to run some plain simulations of the protein-ligand complex. 

The VanillaMDSimulator has lots and lots of options for running simulations in different configurations, however a basic configuration should be ok for us.

In [25]:
VanillaMDSimulator?

Init signature:
VanillaMDSimulator(
    *,
    output_dir: pathlib.Path = 'md',
    debug: bool = False,
    collision_rate: pydantic.types.PositiveFloat = 1,
    openmm_logname: str = 'openmm_log.tsv',
    openmm_platform: asapdiscovery.simulation.simulate.OpenMMPlatform = <OpenMMPlatform.Fastest: 'Fastest'>,
    temperature: pydantic.types.PositiveFloat = 300,
    pressure: pydantic.types.PositiveFloat = 1,
    timestep: pydantic.types.PositiveFloat = 4,
    equilibration_steps: pydantic.types.PositiveInt = 5000,
    reporting_interval: pydantic.types.PositiveInt = 1250,
    num_steps: pydantic.types.PositiveInt = 2500000,
    rmsd_restraint: bool = False,
    rmsd_restraint_atom_indices: list[int] = [],
    rmsd_restraint_type: Optional[str] = None,
    rmsd_restraint_force_constant: pydantic.types.PositiveFloat = 50,
    truncate_steps: bool = True,
    **extra_data: Any,
) -> None
Docstring:      Base class for Simulators.
Init docstring:
Create a new model by parsing and validati

We set up our simulator!

In [10]:
md_simulator = VanillaMDSimulator(
            output_dir="md",
            equilibration_steps=1,
            num_steps=1,
            reporting_interval=1)


And run it! If we were running lots of simulations, we could use `dask-cuda` (see [here](https://docs.rapids.ai/api/dask-cuda/nightly/)) to parallelise it across multiple GPUs. An example of that is given below. 

In [26]:
simulation_results = md_simulator.simulate(
            results,
            use_dask=False)

The simulator makes unique paths for the resulting simulations, we will add some flexibility with output paths in the future. 

In [11]:
print(simulation_results[0].traj_path)

md/SARS2_Mac1A-b27f22555232d2d68273612ffce5a119d6e22526d95ce3eb0db9012632bcdaf6+FHHVXLFEHODNRQ-XCZWEQHLNA-M_alkane-IMNFDUFMRHMDMM-UHFFFAOYNA-N/traj.xtc


In [12]:
print(simulation_results[0].final_pdb_path)

md/SARS2_Mac1A-b27f22555232d2d68273612ffce5a119d6e22526d95ce3eb0db9012632bcdaf6+FHHVXLFEHODNRQ-XCZWEQHLNA-M_alkane-IMNFDUFMRHMDMM-UHFFFAOYNA-N/final.pdb


## Running multiple simulations with dask-cuda

We can pass a series of DockingResults to the VanillaMDSimulator and have `dask-cuda` parallelize work over available GPU resource. For this we will need a LocalCUDACluster.

**NOTE** dask_cuda is not available for `MacOS` computers.

In [30]:
# create a dask_cuda LocalCUDACluster
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)

ModuleNotFoundError: No module named 'dask_cuda'

In [31]:
# asapdiscovery provides a convenience function to do this
from asapdiscovery.data.util.dask_utils import DaskType, make_dask_client_meta
client = make_dask_client_meta(DaskType.LOCAL_GPU)

ImportError: dask_cuda is not installed, please install with `pip install dask_cuda`

We can see by in

In [27]:
VanillaMDSimulator.simulate?

Signature:
VanillaMDSimulator.simulate(
    self,
    docking_results: list[asapdiscovery.docking.docking.DockingResult],
    use_dask: bool = False,
    dask_client=None,
    failure_mode=<FailureMode.SKIP: 'skip'>,
    **kwargs,
) -> pandas.core.frame.DataFrame
Docstring: <no docstring>
File:      ~/Desktop/asap/asapdiscovery/asapdiscovery-simulation/asapdiscovery/simulation/simulate.py
Type:      function